In [1]:
import pandas as pd

def read_and_prepare_data(filepath):
    """อ่านไฟล์และเตรียมข้อมูลเบื้องต้น"""
    df = pd.read_csv(filepath)
    df['วันที่เกิดเหตุ'] = pd.to_datetime(df['วันที่เกิดเหตุ'].astype(str).str.strip(), errors='coerce')
    df['จำนวนอุบัติเหตุ'] = 1
    return df

def group_accident_counts(df):
    """รวมจำนวนอุบัติเหตุตาม วันที่ และ จังหวัด"""
    return (
        df.groupby(['วันที่เกิดเหตุ', 'จังหวัด'])['จำนวนอุบัติเหตุ']
        .sum()
        .reset_index()
    )

def fill_missing_dates_provinces(grouped_df):
    """เติมวันที่-จังหวัดที่ไม่มีเหตุการณ์ให้มีจำนวนอุบัติเหตุ = 0"""
    all_dates = pd.date_range(
        start=grouped_df['วันที่เกิดเหตุ'].min(),
        end=grouped_df['วันที่เกิดเหตุ'].max()
    )
    all_provinces = grouped_df['จังหวัด'].unique()
    full_index = pd.MultiIndex.from_product(
        [all_dates, all_provinces],
        names=['วันที่เกิดเหตุ', 'จังหวัด']
    )
    complete_df = (
        grouped_df.set_index(['วันที่เกิดเหตุ', 'จังหวัด'])
        .reindex(full_index, fill_value=0)
        .reset_index()
    )
    return complete_df.sort_values(by='วันที่เกิดเหตุ')

def save_to_csv(df, output_path):
    """บันทึกไฟล์ CSV"""
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

อ่านและเตรียมข้อมูล

In [3]:
input_csv = '../output/accident2024_Damage_provinces.csv'
df = read_and_prepare_data(input_csv)
df.head()

/var/folders/mh/smhv27qx1n769y841db29nmr0000gn/T/ipykernel_20138/3102555854.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['วันที่เกิดเหตุ'] = pd.to_datetime(df['วันที่เกิดเหตุ'].astype(str).str.strip(), errors='coerce')


,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,ACC_CODE,หน่วยงาน,สายทางหน่วยงาน,รหัสสายทาง,สายทาง,...,รถบรรทุกมากกว่า10ล้อ,รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็ก,รวมจำนวนผู้บาดเจ็บ,มูลค่าความเสียหาย,จำนวนอุบัติเหตุ
0,2024.0,2024-01-01,0:12,06/18/2024,14:23,9701543.0,กรมทางหลวง,ทางหลวง,4164,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,62536,1
1,2024.0,2024-01-01,0:30,01-01-24,12:13,8901889.0,กรมทางหลวง,ทางหลวง,106,ลี้ - ม่วงโตน,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,653759,1
2,2024.0,2024-01-01,0:30,01-02-24,4:18,8902334.0,กรมทางหลวง,ทางหลวง,1143,น้ำคลาด - ปางหมิ่น,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,423925,1
3,2024.0,2024-01-01,0:30,01-01-24,11:40,8902375.0,กรมทางหลวง,ทางหลวง,3390,หนองรี - บ่อยาง,...,0.0,0.0,0.0,2.0,0.0,1.0,1.0,2.0,406010,1
4,2024.0,2024-01-01,0:30,01-01-24,19:07,8902399.0,กรมทางหลวง,ทางหลวง,221,แยกการช่าง - เชิงบันไดเขาพระวิหาร,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,454972,1


รวมจำนวนอุบัติเหตุ

In [4]:
grouped_df = group_accident_counts(df)
grouped_df.head()

,วันที่เกิดเหตุ,จังหวัด,จำนวนอุบัติเหตุ
0,2024-01-01,กระบี่,1
1,2024-01-01,กรุงเทพมหานคร,8
2,2024-01-01,กาญจนบุรี,5
3,2024-01-01,กาฬสินธุ์,2
4,2024-01-01,กำแพงเพชร,1


เติมข้อมูลวันที่-จังหวัดที่ไม่มีเหตุการณ์

In [5]:
complete_df = fill_missing_dates_provinces(grouped_df)
complete_df.head()

,วันที่เกิดเหตุ,จังหวัด,จำนวนอุบัติเหตุ
0,2024-01-01,กระบี่,1
55,2024-01-01,หนองคาย,1
54,2024-01-01,สุโขทัย,1
53,2024-01-01,สุรินทร์,1
52,2024-01-01,สุราษฎร์ธานี,9


บันทึกไฟล์ CSV

In [7]:
output_csv = '../output/accident2024_sum_accident.csv'
save_to_csv(complete_df, output_csv)
print(f"✅ Done! Output saved to: {output_csv}")

✅ Done! Output saved to: ../output/accident2024_sum_accident.csv


บันทึกไฟล์ Excel

In [11]:
output_excel = '../output/accident2024_sum_accident.xlsx'
complete_df['วันที่เกิดเหตุ'] = complete_df['วันที่เกิดเหตุ'].dt.date
complete_df.to_excel(output_excel, index=False, engine='openpyxl')
print(f"✅ Done! Output saved to: {output_excel}")

✅ Done! Output saved to: ../output/accident2024_sum_accident.xlsx
